**Importing Modules**

In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

from keras import layers

from keras import models

from keras.layers import (Input, Dense, Activation, ZeroPadding2D,
BatchNormalization, Flatten, Conv2D, concatenate, Lambda)

from keras.layers import (AveragePooling2D, MaxPooling2D, Dropout,
GlobalMaxPooling2D, GlobalAveragePooling2D)

from keras.models import Model, load_model
from keras import regularizers, optimizers

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical

print(os.listdir(r'C:\Users\kszyh\Desktop\dane'))
print(os.listdir(r'C:\Users\kszyh\Desktop\dane\self driving car training data\data'))

['archive.zip', 'data', 'img_zaznaczone', 'self driving car training data']
['driving_log.csv', 'IMG']


In [2]:
from tqdm import tqdm

**Reading Driving Data from driving_log.csv**

In [3]:
path = r'C:\Users\kszyh\Desktop\dane\self driving car training data\data'
path = os.path.join(path,'driving_log.csv')

data_frame = pd.read_csv(path)
center = data_frame[data_frame.columns[0]].values
left = data_frame[data_frame.columns[1]].values
right = data_frame[data_frame.columns[2]].values
steering = data_frame[data_frame.columns[3]].values

no_of_examples = len(steering)
print(no_of_examples)

8036


**Helper Function for data Augmentation**

In [4]:
def random_flip(image, steering_angle):
    
    image = cv2.flip(image, 1)
    steering_angle = -steering_angle
    
    return image, steering_angle

**Collecting the Training Data**

In [5]:
train_x = []
train_y = []

img_folder = r'C:\Users\kszyh\Desktop\dane\data\IMG'
img_folder = r'C:\Users\kszyh\Desktop\dane\img_zaznaczone\IMG'
stear_adjust_factor = 0.2
IMAGE_HEIGHT = 100 
IMAGE_WIDTH = 100

for i in tqdm(range(no_of_examples)):
    
    for choice in range(3):
        
        if choice == 0: #Center
            img = cv2.imread(os.path.join(img_folder,center[i].split('IMG/')[1]))
            steering_angle = steering[i]

        elif choice == 1: #Left
            img = cv2.imread(os.path.join(img_folder,left[i].split('IMG/')[1]))
            steering_angle = steering[i] + stear_adjust_factor

        elif choice == 2: #Right
            img = cv2.imread(os.path.join(img_folder,right[i].split('IMG/')[1]))
            steering_angle = steering[i] - stear_adjust_factor
        
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:,:,1]
        img = cv2.resize(img,(IMAGE_HEIGHT,IMAGE_WIDTH))
        
        train_x.append(img)
        train_y.append(steering_angle)
        
        flip_img,steering_angle = random_flip(img,steering_angle)
                
        train_x.append(flip_img)
        train_y.append(steering_angle)
        

train_x = np.array(train_x)
train_x = np.reshape(train_x,[train_x.shape[0],train_x.shape[1],train_x.shape[2],1])

train_y = np.array(train_y)
train_y = np.reshape(train_y,[train_y.shape[0],1])

print(train_x.shape)
print(train_y.shape)

100%|█████████████████████████████████████████████████████████████████████████████| 8036/8036 [01:03<00:00, 126.02it/s]


(48216, 100, 100, 1)
(48216, 1)


**Splitting the Training Data into Train and Test sets**

In [6]:
x_train,x_test,y_train,y_test = train_test_split(train_x,train_y,random_state=42,test_size=.20)
print(type(x_test), x_test.shape)
print(type(y_test), y_test.shape)

<class 'numpy.ndarray'> (9644, 100, 100, 1)
<class 'numpy.ndarray'> (9644, 1)


In [10]:
x_test[0].shape

(100, 100, 1)

In [11]:
import tensorflow as tf

**Neural Network Architecture**

In [12]:
def model(height,width):
        
    x_input = Input(shape=(height,width,1))
    
    x = Lambda(lambda x: x/127.5-1.0)(x_input)
    
    x = Conv2D(32,(3,3),activation='relu',padding='same')(x_input)
    
    x = Conv2D(32,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Conv2D(64,(3,3),activation='relu',padding='same')(x)
    
    x = Conv2D(64,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)
    
    x = Flatten()(x)
    x = Dropout(0.5)(x)
       
    x = BatchNormalization()(x)
    x = Dense(512)(x)
    x = Dense(256)(x)
    x = Dense(64)(x)
    x = Dense(1)(x)
    
    model = Model(inputs=x_input,outputs=x,name='model')
    
    return model

with tf.device('/CPU:0'):
    
    model = model(IMAGE_HEIGHT,IMAGE_WIDTH)
    print(model.summary())

    opt = optimizers.Adam(lr=0.0001)
    model.compile(loss='mse',
                 optimizer=opt,
                 metrics=['mse'])

    hist = model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=32,epochs=10)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 100, 100, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 50, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 50, 50, 64)        36928     
                                                             

E:\conda\envs\keras\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1206/1206 [==============================] - 490s 406ms/step - loss: 0.2238 - mse: 0.2238 - val_loss: 0.0213 - val_mse: 0.0213
Epoch 2/10
1206/1206 [==============================] - 484s 402ms/step - loss: 0.0285 - mse: 0.0285 - val_loss: 0.0245 - val_mse: 0.0245
Epoch 3/10
1206/1206 [==============================] - 490s 406ms/step - loss: 0.0302 - mse: 0.0302 - val_loss: 0.0230 - val_mse: 0.0230
Epoch 4/10
1206/1206 [==============================] - 494s 410ms/step - loss: 0.0282 - mse: 0.0282 - val_loss: 0.0236 - val_mse: 0.0236
Epoch 5/10
1206/1206 [==============================] - 495s 411ms/step - loss: 0.0266 - mse: 0.0266 - val_loss: 0.0241 - val_mse: 0.0241
Epoch 6/10
1206/1206 [==============================] - 545s 452ms/step - loss: 0.0249 - mse: 0.0249 - val_loss: 0.0227 - val_mse: 0.0227
Epoch 7/10
1206/1206 [==============================] - 576s 477ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.0213 - val_mse: 0.0213
Epoch 8/10
1206/1206 [=======================

**Saving the Model**

In [13]:
model.save('model_sterowanie.h5')